# Работа с файлами данных

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')
import pandas as pd

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

v 0.2 05.10.22

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [ ]:
import json
import pandas as pd
import pickle
from bs4 import BeautifulSoup

## Лабораторная работа №3.1

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [5]:
import json

# Открытие файла
with open("contributors_sample.json") as f:
    data = json.load(f)

# Вывод информации о первых 3 пользователях
for contributor in data[:3]:
    print(f"Username: {contributor['username']}")
    print(f"name: {contributor['name']}")
    print(f"adress: {contributor['address']}")
    print()


Username: uhebert
name: Lindsey Nguyen
adress: 01261 Cameron Spring
Taylorfurt, AK 97791

Username: vickitaylor
name: Cheryl Lewis
adress: 66992 Welch Brooks
Marshallshire, ID 56004

Username: sheilaadams
name: Julia Allen
adress: Unit 1632 Box 2971
DPO AE 23297



1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [4]:
import re

# создаем множество для хранения уникальных доменов
domains = set()

# проходимся по каждому пользователю и извлекаем домен из его адреса электронной почты
for user in data:
    email = user['mail']
    domain = re.search(r'@([\w.]+)', email).group(1)
    domains.add(domain)

# выводим список уникальных доменов
print(list(domains))


['gmail.com', 'yahoo.com', 'hotmail.com']


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [7]:
def get_contributor_by_username(username):
    for contributor in data:
        if contributor['username'] == username:
            return contributor
    raise ValueError(f'Contributor with username "{username}" not found')
username = 'octocat'
try:
    contributor = get_contributor_by_username(username)
    print(contributor)
except ValueError as e:
    print(e)


Contributor with username "octocat" not found


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [8]:
genders = {}
for contributor in data:
    if contributor['sex'] in genders:
        genders[contributor['sex']] += 1
    else:
        genders[contributor['sex']] = 1
print(genders)


{'F': 2136, 'M': 2064}


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [15]:
contributors = pd.DataFrame(columns=['id', 'username', 'sex'])
for contributor in contributors:
    contributors = contributors.append({
        'id': contributor['id'],
        'username': contributor['login'],
        'sex': contributor['sex']
    }, ignore_index=True)
print(contributors.head())


TypeError: string indices must be integers, not 'str'

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует?

In [16]:
import pandas as pd
import numpy as np
recipes = pd.read_csv('recipes_sample.csv')
merged = recipes.merge(contributors, how='left', left_on='submitter_id', right_on='id')
merged['sex'] = merged['sex'].fillna('unknown')
unknown_count = np.sum(merged['sex'] == 'unknown')
print(f'Information about {unknown_count} people is missing in JSON file')


KeyError: 'submitter_id'

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
job_people = defaultdict(list)
for user in contributors:
  for job in user['jobs']:
    job_people[job].append(user['username'])


2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
with open(r'/content/drive/MyDrive/datasets/tobd/3_data_files/job_people.json', 'w', encoding='utf8') as f:
  json.dump(job_people, f, indent=2)

In [ ]:
with open(r'/content/drive/MyDrive/datasets/tobd/3_data_files/job_people.pickle', 'wb') as f:
  pickle.dump(job_people, f)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open(r'/content/drive/MyDrive/datasets/tobd/3_data_files/job_people.pickle', 'rb') as f:
  job_people = pickle.load(f)

job_people

## Лабораторная работа №3.2

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`